In [1]:
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
from decouple import config

In [2]:
# instantiate the API and set the interval to weekly data
dh = YahooFinanceAPI(Interval.DAILY)

# get datime of yesterday at 00:00:00
yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
now = datetime.strptime(yesterday, "%Y, %m, %d")
then = datetime(2022, 3, 12)

In [3]:
import os
os.getcwd()

'C:\\Users\\C'

In [5]:
#dbuname = 'Christoph'
#dbpwd = 'dbyahoo1'
#host = 'dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com'
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = '5432'
dbname = 'dbyf1'
print(dbuname)
print(dbpwd)
print(host)

Christoph
dbyahoo1
dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com


In [6]:
labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq' }

In [7]:
def fetch_load_data(instrument, start, end):
    #fetch data
    df_data = dh.get_ticker_data(instrument, start, end)
    table_name = 'table_'+labels[instrument]
    print(table_name)
    
    #load data
    conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
    engine = create_engine(conn_string)
    df_data.to_sql(table_name, conn_string, if_exists = 'replace', index=False)    
    

In [8]:
inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']

for ins in inv_instruments:
    fetch_load_data(ins, then, now)

table_msci_world
table_euro_stoxx
table_smi
table_nasdaq
table_gold
table_silver


In [ ]:
# Fetch Data from database (nur für Kontrolle)

In [108]:
try: 
    conn = psycopg2.connect("host=dbyf1.c8vnebtzzltk.us-east-1.rds.amazonaws.com dbname=dbyf1 user=Christoph password=dbyahoo1")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)

In [186]:
try: 
    cur.execute("SELECT * FROM table_nasdaq;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(datetime.datetime(2022, 3, 14, 0, 0), 119.080002, 119.300003, 116.5, 116.599998, 116.599998, 37605)
(datetime.datetime(2022, 3, 15, 0, 0), 115.120003, 119.120003, 114.540001, 119.0, 119.0, 64273)
(datetime.datetime(2022, 3, 16, 0, 0), 121.18, 122.580002, 120.779999, 122.019997, 122.019997, 56915)
(datetime.datetime(2022, 3, 17, 0, 0), 123.239998, 123.300003, 121.800003, 122.519997, 122.519997, 21375)
(datetime.datetime(2022, 3, 18, 0, 0), 123.459999, 125.959999, 123.339996, 125.940002, 125.940002, 27809)
(datetime.datetime(2022, 3, 21, 0, 0), 126.620003, 127.459999, 125.68, 126.279999, 126.279999, 38657)
(datetime.datetime(2022, 3, 22, 0, 0), 127.660004, 129.759995, 126.940002, 129.300003, 129.300003, 80574)
(datetime.datetime(2022, 3, 23, 0, 0), 129.539993, 130.0, 128.619995, 129.440002, 129.440002, 41454)


In [ ]:
cur.close()
conn.close()